# Image Segmentation using Graph Cut

**Project Overview**  
The goal of this project is to segment an image into foreground and background regions using Graph Cut, a graph-based algorithm that minimizes energy to find an optimal segmentation. This technique is widely used in interactive segmentation tasks, such as separating an object from its background.

**Goal**

The goal of graph cut is to segment an image into regions by modeling the image as a graph, where pixels (or groups of pixels) are represented as nodes, and edges represent relationships between pixels (such as intensity differences or other features). The algorithm works by finding a set of edges (a cut) that completely separates the graph into two disjoint regions, with one labeled as the foreground and the other as the background. The optimal cut minimizes a cost function, ensuring that the separation respects both the similarity between pixels and the constraints provided by user-defined seeds or priors.

**Ford-Fulkerson algorithm**

In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

class GraphCutSegmentation:
    def __init__(self, image, foreground_seeds, background_seeds):
       
        self.image = image
        self.foreground_seeds = foreground_seeds
        self.background_seeds = background_seeds
        self.rows, self.cols = image.shape
        self.graph = None
        self.source = 's'
        self.sink = 't'

    def initialize_graph(self):
        """
        Initialize the graph as a dictionary to store edges and their capacities.
        """
        self.graph = {}

        # Add source and sink nodes
        self.graph[self.source] = {}
        self.graph[self.sink] = {}

        # Add nodes for each pixel
        for i in range(self.rows):
            for j in range(self.cols):
                self.graph[(i, j)] = {}

    def add_edge(self, node1, node2, capacity):
        """
        Add an edge between two nodes with a given capacity, and initialize reverse edges with zero capacity.
        """
        if node1 not in self.graph:
            self.graph[node1] = {}
        if node2 not in self.graph:
            self.graph[node2] = {}
        
        self.graph[node1][node2] = capacity
        
        # Initialize reverse edge with zero capacity
        if node2 not in self.graph[node1]:
            self.graph[node2][node1] = 0

    def build_graph(self):
        """
        Build the graph by connecting pixels with edge weights and adding source/sink edges.
        """
        # Add source and sink edges
        for (x, y) in self.foreground_seeds:
            self.add_edge(self.source, (x, y), float('inf'))
        for (x, y) in self.background_seeds:
            self.add_edge((x, y), self.sink, float('inf'))

        # Add edges between neighboring pixels
        for i in range(self.rows):
            for j in range(self.cols):
                if i < self.rows - 1:  # Vertical edge
                    weight = self.compute_edge_weight((i, j), (i + 1, j))
                    self.add_edge((i, j), (i + 1, j), weight)
                    self.add_edge((i + 1, j), (i, j), weight)
                if j < self.cols - 1:  # Horizontal edge
                    weight = self.compute_edge_weight((i, j), (i, j + 1))
                    self.add_edge((i, j), (i, j + 1), weight)
                    self.add_edge((i, j + 1), (i, j), weight)

    def compute_edge_weight(self, pixel1, pixel2):
        intensity_diff = abs(int(self.image[pixel1]) - int(self.image[pixel2]))
        return np.exp(-intensity_diff / 10.0)
    

    def bfs(self, residual_graph, source, sink, parent):
        visited = set()
        queue = [source]
        visited.add(source)

        while queue:
            u = queue.pop(0)

            for v in residual_graph[u]:
                if v not in visited and residual_graph[u][v] > 0:  # Positive capacity
                    queue.append(v)
                    visited.add(v)
                    parent[v] = u
                    if v == sink:
                        return True
        return False

    def ford_fulkerson(self):
        # Initialize the residual graph with reverse edges
        residual_graph = {u: v.copy() for u, v in self.graph.items()}
        for u in self.graph:
            for v in self.graph[u]:
                if v not in residual_graph:
                    residual_graph[v] = {}
                if u not in residual_graph[v]:
                    residual_graph[v][u] = 0

        parent = {}
        max_flow = 0

        while self.bfs(residual_graph, self.source, self.sink, parent):
            # Find minimum capacity in the augmenting path
            path_flow = float('inf')
            s = self.sink
            while s != self.source:
                path_flow = min(path_flow, residual_graph[parent[s]][s])
                s = parent[s]

            # Update residual capacities
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual_graph[u][v] -= path_flow
                residual_graph[v][u] += path_flow
                v = parent[v]

            max_flow += path_flow

        # Find reachable nodes
        reachable = set()
        queue = [self.source]
        while queue:
            u = queue.pop(0)
            reachable.add(u)
            for v in residual_graph[u]:
                if v not in reachable and residual_graph[u][v] > 0:
                    queue.append(v)
        return max_flow, reachable


    def segment(self):
        """
        Perform segmentation using the graph cut algorithm.

        Returns:
        - segmentation: Binary mask of the segmented image.
        """
        self.initialize_graph()
        self.build_graph()
        _, reachable = self.ford_fulkerson()

        # Create segmentation mask
        segmentation = np.zeros(self.image.shape, dtype=np.uint8)
        for i in range(self.rows):
            for j in range(self.cols):
                if (i, j) in reachable:
                    segmentation[i, j] = 255

        return segmentation

# Load a grayscale image
image = cv2.imread('../Object-Detection/Template-Matching/tumor.png', cv2.IMREAD_GRAYSCALE)
image_resize = cv2.resize(image, (50,50))



# Define foreground and background seeds
foreground_seeds = [(18, 35), (19, 36)]
background_seeds = [(1, 0), (4, 0)]

# Perform Graph Cut Segmentation
graph_cut = GraphCutSegmentation(image_resize, foreground_seeds, background_seeds)
segmentation = graph_cut.segment()

# Display results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image_resize, cmap='gray')
plt.subplot(1, 2, 2)
plt.title("Segmented Image")
plt.imshow(segmentation, cmap='gray')
plt.show()

: 